In [3]:
import folium
import folium.plugins as fp
import json
import pandas as pd

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
f_path='C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

In [9]:
df_stop = pd.read_excel('../data/공공자전거 대여소 정보.xlsx', encoding='utf-8')
df_stop.head()
# df_stop['구분'].unique()

,구분,대여소번호,대여소명,위도,경도,거치대수
0,마포구,101,101. (구)합정동 주민센터,37.549561,126.905754,5
1,마포구,102,102. 망원역 1번출구 앞,37.556000,126.910454,20
2,마포구,103,103. 망원역 2번출구 앞,37.554951,126.910835,14
3,마포구,104,104. 합정역 1번출구 앞,37.550629,126.914986,13
4,마포구,105,105. 합정역 5번출구 앞,37.550007,126.914825,5


In [14]:
df_stop_total = df_stop.groupby('구분')['거치대수'].sum().to_frame()
df_stop_total.head()

,거치대수
구분,
강남구,1210
강동구,773
강북구,451
강서구,1014
관악구,652


In [47]:
map = folium.Map(location = [37.5202, 126.975], zoom_start=11, width=750, height=750)
mc = fp.MarkerCluster(control=False)
map.add_child(mc)
geo_path = '../data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map.choropleth(geo_data = geo_str, \
               data=df_stop_total, \
               columns=[df_stop_total.index, '거치대수'], \
               fill_color='RdBu', \
               key_on='feature.id')
for n in df_stop.index:
    if pd.notnull(df_stop['위도'][n]):
        string = f'{df_stop["대여소명"][n]}<br>{df_stop["거치대수"][n]}'
        popup = folium.Popup(string, min_width=130, max_width=130)
        folium.Marker([df_stop['위도'][n], df_stop['경도'][n]], \
                      icon=folium.Icon(color='blue', icon='ok'), \
                      popup=popup).add_to(mc)
#                       popup='%s<br>%d'%(df_stop['대여소명'][n], df_stop['거치대수'][n])).add_to(mc)
map